-*- coding: utf-8 -*-

"""

Created on Thu Aug 23, 2020

@author: Rohitashwa

@Credits: loren ( https://github.com/jjeffus/gofundme)

"""

In [ ]:
# !pip install selenium
# !pip install bs4

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import numpy as np
import pandas as pd
import time

## Scrape HomePage to identify all fundraiser wepage URLs

**NOTE:**
The following categories' webpage URLs do not follow the format specifies in the code:
- Animals: 
    Url: ~animals-fundraiser~ -> animal-fundraiser
    
- NonProfits:
    Url: ~nonprofits-fundraiser~ -> charity-fundraiser
    
**RESCRAPE:**
The following categories need to be rescraped due to insufficient (<1,000) fundraiser count:

In [ ]:
url = 'https://www.gofundme.com/discover'
driver = webdriver.Chrome('C:/Users/rohit/OneDrive/Desktop/IIMa/chromedriver_win32/chromedriver.exe')
driver.get(url)

In [ ]:
for elem in driver.find_elements_by_link_text('Show all categories'):
        try:
            elem.click()
            print('Succesful click')
        except:
            print('Unsuccesful click')

In [ ]:
source = driver.page_source

driver.close()

soup = BeautifulSoup(source, 'lxml')
cat_containers = soup.findAll("div", {"class": "section-categories grid-x small-up-2 medium-up-3 large-up-6"})
categories = cat_containers[0].text #contains category names for this section
categories = categories.splitlines()
categories = list(filter(None, categories))

In [ ]:
more_containers = soup.findAll("div", {"class": "section-categories grid-x small-up-2 medium-up-3 large-up-6 js-more-categories"})
more_cats = more_containers[0].text
more_cats = more_cats.splitlines()
more_cats = list(filter(None, more_cats))
all_cats = categories + more_cats # should be len(all_cats) == 18
# all_cats

In [ ]:
#make url_categories
categories_urls = list(map(lambda x:{x:'https://www.gofundme.com/discover/{}-fundraiser'.format(x.lower())}, all_cats))
categories_urls

In [ ]:
# mydf = pd.read_csv('data/GFM_url_list.csv', sep = '\t')
# headers = ["Url", "Category","Position", "Title", "Location", "Organiser","Amount_Raised", "Goal", "Duration", "Text", "Image"]
# mydf = mydf.reindex(columns = headers)
# mydf.head()

# Process to extract individual gofundme urls from an individual category

In [ ]:
def extract_urls_from_categories(url, MoreGFMclicks = 5):
    # eg. url = 'https://www.gofundme.com/discover/medical-fundraiser'
    category = url.split("/")[-1]
    driver = webdriver.Chrome('C:/Users/rohit/OneDrive/Desktop/IIMa/chromedriver_win32/chromedriver.exe')
    driver.get(url)
    
    for i in range(MoreGFMclicks):
        if(len(driver.find_elements_by_link_text('Show More'))==0):
            break
        for elem in driver.find_elements_by_link_text('Show More'):
            try:
                elem.click()
                print(category+' Succesful click %s' %(i+1))#make this more useful- say what category it is e.g. url.get_category()
            except:
                print(category+ ' Unsuccesful click %s' %(i+1))
                
            sleep(6) #longer delay - more succesful
            # 5 seconds to ensure GFM doesn't ban us
            
    source = driver.page_source
        
    driver.close()
    
    soup = BeautifulSoup(source, 'lxml')
    with open("data/webpages/sample1_"+category+".html \n", "w", encoding='utf-8') as file:
        file.write(str(soup))
    
    containers = soup.findAll("div", {"class": "cell grid-item small-6 medium-4 js-fund-tile"})
    print("Number of Fundraisers found:"+str(len(containers)))
    temp_url = {}
    i = 1
    
    for container in containers:
        for link in container.find_all('a'):
            temp_url[link.get('href')] = i
            i = i + 1 

    temp_url = {k: ((v // 2) - 1) // 3  for k, v in temp_url.items()} #


    return(temp_url)

# Generate lists of list of URL per category

In [ ]:
# Redo Animals & Business

target_categories = [
#     {'Medical': 'https://www.gofundme.com/discover/medical-fundraiser'}-
#     ,{'Memorial': 'https://www.gofundme.com/discover/memorial-fundraiser'}-
#     ,{'Emergency': 'https://www.gofundme.com/discover/emergency-fundraiser'}-
#     ,{'Charity': 'https://www.gofundme.com/discover/charity-fundraiser'}-
#     ,{'Education': 'https://www.gofundme.com/discover/education-fundraiser'}-
#     ,{'Animals': 'https://www.gofundme.com/discover/animal-fundraiser'}-
#     ,{'Business': 'https://www.gofundme.com/discover/business-fundraiser'}-
#     ,{'Community': 'https://www.gofundme.com/discover/community-fundraiser'}-
#     ,{'Competition': 'https://www.gofundme.com/discover/competition-fundraiser'}-
#     ,{'Creative': 'https://www.gofundme.com/discover/creative-fundraiser'}-
#     ,{'Event': 'https://www.gofundme.com/discover/event-fundraiser'}-
#     ,{'Faith': 'https://www.gofundme.com/discover/faith-fundraiser'}-
#     ,{'Family': 'https://www.gofundme.com/discover/family-fundraiser'}-
#     ,{'Newlywed': 'https://www.gofundme.com/discover/newlywed-fundraiser'}-
#     ,{'Sports': 'https://www.gofundme.com/discover/sports-fundraiser'}-
#     ,{'Travel': 'https://www.gofundme.com/discover/travel-fundraiser'}-
#     ,{'Volunteer': 'https://www.gofundme.com/discover/volunteer-fundraiser'}-
#     ,{'Wishes': 'https://www.gofundme.com/discover/wishes-fundraiser'}-
]

target_categories

## Sequential Mode

In [ ]:
def list_urls(categoryURLs, moreGFMclicks = 5):
    GFM_urls = []
    for url_pair in categoryURLs:
        category = list(url_pair.keys())[0]
        url = list(url_pair.values())[0]
        print("Currently Scraping: "+url)
        GFM_urls.append([extract_urls_from_categories(url, MoreGFMclicks = moreGFMclicks), category])#get category from categories_urls
    print("All done!")
    return(GFM_urls)


GFM_Urls = list_urls(target_categories,0)
GFM_Urls

In [ ]:
return_value = [(list_urlsV2((item,90))) for item in target_categories]

# flatten list

GFM_Urls_long = []
for k in return_value:
#     print(k[0][0])
    GFM_Urls_long.extend(list(k[0][0]))

mydf = {
    "Url": GFM_Urls_long,
    "Category": ["Education"]*len(GFM_Urls_long)
}
mydf = pd.DataFrame(_)

## Multithreaded Mode

In [ ]:
def list_urlsV2(params):
    categoryURLs = params[0]
    moreGFMclicks = params[1]
    GFM_urls = []
    for url_pair in categoryURLs:
        category = url_pair
        url = categoryURLs[url_pair]
        print("Currently Scraping: "+url)
        t_init = time.time()
        GFM_urls.append([extract_urls_from_categories(url, MoreGFMclicks = moreGFMclicks), category])#get category from categories_urls
        t_fin = time.time() - t_init
    print("All done for "+category+" category in "+str(t_fin)+"m-sec!")
    return(GFM_urls)

In [ ]:
import concurrent.futures

t0= time.time()

GFM_Urls_long = []

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(list_urlsV2, (item,90)) for item in target_categories]
    return_value = [f.result() for f in futures]

    rowCount = 1
    for k in return_value:
        GFM_Urls_long.extend(k)
#         rowCount = rowCount + batch_size

t1= time.time() -t0
print("Time-Elapsed total: "+str(t1))
print(len(GFM_Urls_long))
# GFM_Urls_long

In [ ]:
GFM_Urls_long

In [ ]:
#flatten list
mydf = pd.DataFrame(columns = ["Url", "Category","Position"])
for cat in GFM_Urls_long:
    
    temp_val = np.array(list(cat[0].values()))
    temp_key = np.array(list(cat[0].keys()))
    temp_category = np.repeat(cat[1], len(cat[0]))
    
    temp_df = pd.DataFrame(columns = ["Url", "Category", "Position"])
    temp_df["Position"] = temp_val
    temp_df["Category"] = temp_category
    temp_df["Url"] = temp_key
    
    mydf = mydf.append(temp_df, ignore_index = True)
mydf

## Save to file

In [ ]:
fileName = 'data/GFM_url_list.csv'
# mydf.to_csv(fileName, sep='\t', index = False, header = False, mode = 'a')

In [ ]:
mydf = pd.read_csv(fileName, sep='\t')
mydf

**NOTE:**
The data is split over the following files:
- data/GFM_url_7k.csv(Scraped; uploaded)
- data/GFM_url2_3k.csv
- data/GFM_url3_2k.csv

## Combining all CSVs 

In [ ]:
# import pandas as pd
# import numpy as np

In [ ]:
# fileList = [
#     "data/GFM_url_business_1k.csv",
#     "data/GFM_url_emergency_1k.csv",
#     "data/GFM_url_medical_1k.csv",
#     "data/GFM_url_memorialEducation_1k.csv",
#     'data/GFM_url_volunteer_1k.csv',
#     "data/GFM_url_competition_1k.csv"
# ]

In [ ]:
# final = pd.DataFrame()
# for item in fileList:
#     x = pd.read_csv(item,sep = '\t')
#     print(len(x))
#     final = pd.concat([final,x], ignore_index = True)
    
# final.drop(['Unnamed: 0'],axis = 1, inplace = True)
# final.tail()

In [ ]:
# final.to_csv('data/GFM_url_compiled.csv', sep='\t', index = False)

In [ ]:
# pd.read_csv('data/GFM_url_compiled.csv', sep='\t')